<a href="https://colab.research.google.com/github/hatahataDev/SDGsproject/blob/master/Machine_STI2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive # グーグルドライブをマウントさせるコード。
drive.mount('/content/drive')
# dir="drive/My Drive/ohata20/text" # drive内に入れたtextのパスを書く。
data_dir="drive/My Drive/ohata20/STI2.csv" # CSVファイルのパス
predict_dir="drive/My Drive/ohata20/needpredict.csv"# 予測データのパス

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
"""
入力用データ成型機
以下　Goal4の公式文
By 2030, ensure that all young people and the majority (both men and women) adults have literacy and basic computing skills.
By 2030, education for sustainable development and sustainable lifestyles, human rights, gender equality, promotion of peace and non-violent culture, global citizenship, sustainable development of cultural diversity and culture Ensure that all learners have the knowledge and skills necessary to promote sustainable development through education in understanding their contributions.
Establish and improve educational facilities that are child, disability and gender sensitive and provide a safe, non-violent, inclusive and effective learning environment for all.
By 2020, developed countries such as vocational training, information and communication technology (ICT), technology, engineering and science programs for developing countries, especially least developed and small island developing countries, and African countries Significantly increase the number of higher education scholarships worldwide in other developing countries.

"""
# https://qiita.com/shimayu22/items/4fc6d01a4bf0eef909a0 参考記事
# https://qiita.com/shimayu22/items/4fc6d01a4bf0eef909a0
# 課題～ハイパーパラメータをいじる、あるいはデータの抜き出し、整形の制度を上げる。
# 数値のベクトル化のところのアルゴリズムが少し不自然な気がするからそこの修正
# ネット回線ないとコード動きません。
# ネットのSDGs用語も辞書に足し合わせるべし←ネットに乗っている用語は重要度を上げるなどの改良の余地あり。
# 大文字を小文字にする処理や記号を抜き取る処理を追加する。
import os
import csv
import nltk
from sklearn.model_selection import train_test_split

nltk.download('punkt') # nltkを使うためのソフト群
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

csv_file = open(data_dir,'r') # 学習データ
p_n_list = input("予測したい文章を入力してください。")
# p_csv_file= open(predict_dir,"r") # 予測したいデータ
"""
csvファイルは左から目標番号、タイトル、アブスト、結論の順で入れること。
"""
p_list,t_list,a_list = [],[],[]
# c_list= []

delete= ["a","%","of","in","on","to","for","are","at","[","]"] # 関連性のない語を集めた配列

# p_n_list=[]#予測したいデータのテキストバッグ

# for row in csv.reader(p_csv_file):
# p_n_list.append(row[1])
# p_n_list.append(row[2])

x, y = [], []
count = 0

for row in csv.reader(csv_file):
  # 以下リストを作成
    p_list.append(row[0]) # 取得したい列番号を指定（0始まり）#目標番号をすべて取得
    t_list.append(row[1]) # タイトルをすべて取得
    a_list.append(row[2]) # アブストをすべて取得
    # c_list.append(row[3]) # 結論も入れる

del p_list[0] # 0番目を削除
del t_list[0] # 0番目は削除
del a_list[0] # 0番目は削除。これで_list[i]でそれぞれの列を抜き出すことができる。
# del c_list[0]

# 以下３行　予測したいデータの形態素解析
# p_n_list=p_n_list[0]+p_n_list[1] 
p_n_words = nltk.word_tokenize(p_n_list)
p_n_wards=nltk.pos_tag(p_n_words)# [("単語","品詞")....]の２次元リストをここで生成


p_result = [] # 名詞の数字配列を入れるp_resultを定義
p_N_dic = [] # 名詞のstring型配列を定義
# 以下名詞を抜き出しN_dicに入れ込み
for word,part in p_n_wards: # wordに単語、partに品詞が入っている→このforはタイトル
    if part=="NN" or part=="NNP" or part=="VBP": # 以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete: # さらにdeleteリストに入っている意味のない名詞を削除
        sword=word.lower()
        p_N_dic.append(sword)

Cdic=[] # 各ゴールの単語をすべて入れる。推薦単語用.Cdic[1]→1番目の単語すべてを返す

for i in range(18): # それぞれのゴールの中身を内包した収納配列を生成
  a=[]
  Cdic.append(a) # [[],[],[]......18個]この中に単語をすべて入れていく

dic={} # 辞書で定義
for i,p in enumerate(p_list): # p_listのiをカウンタ変数,ｐを正解ラベル
  j=i+1 # p_list,a_listなどは0番目が存在しないため無理やりjをカウンター変数に置き換え
  if j>=len(p_list): # +1してるから最後の一つだけ存在しないjが入るためここで分岐処理　t_list[j_max]は存在しないためエラー
    break
  N_dic=[] # ここで名詞リストは初期化
  # print(t_list[i])
  t_string = t_list[j]
  t_words = nltk.word_tokenize(t_string)
  t_wards=nltk.pos_tag(t_words)# [("単語","品詞")....]の２次元リストをここで生成
  # print(t_wards)
  a_string = a_list[j]
  a_words = nltk.word_tokenize(a_string)
  a_wards=nltk.pos_tag(a_words)
  """
  c_string = c_list[j]
  c_words = nltk.word_tokenize(c_string)
  c_wards=nltk.pos_tag(c_words)
  """
  for word,part in t_wards: # wordに単語、partに品詞が入っている→このforはタイトル
    if part=="NN" or part=="NNP" or part=="VBP" or part=="NNS" or part=="NNPS": # 以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete: #さらにdeleteリストに入っている意味のない名詞を削除
        sword=word.lower() # 小文字に変換
        N_dic.append(sword)
        Cdic[int(p)].extend([sword]) # swordはstr型なので無理やりリストに変換 extendはlist型しか入らない
  for word,part in a_wards: # wordに単語、partに品詞が入っている→このforはアブスト
    if part=="NN" or part=="NNP" or part=="VBP" or part=="NNS" or part=="NNPS": # 以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete:
        sword=word.lower()
        N_dic.append(sword)
        Cdic[int(p)].extend([sword]) # pはstrなのでキャスト
  """
  for word,part in c_wards: #wordに単語、partに品詞が入っている→このforはアブスト
    if part=="NN": #以下if文は名詞のみをN_dicというリストに格納
      N_dic.append(word)
    if part=="NNP":
      N_dic.append(word)
  """
  print(N_dic)# すべてN_dicにまとめてしまう。

  result=[]
  for word in N_dic:
    word = word.strip()  
    if word == "": continue
    if not word in dic: # 未登録の場合
      dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
      num = count
      count +=1
    # print(num,word)  # 数字と単語を表示
    else:
      num=dic[word] # 数字を辞書で調べる
    result.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している
  x.append(result)  # リストを配列 ｘ に追加 x = [ [ 10, 11, 12, 3, 8, 13,  … 38 ] , [ 39, 40, 12, 16, 19, … ,56 ] , ….  ,  [ 1200, 1504, 3, 15, … 3300 ] ] 
  p_p=int(p)  # pは文字型で入っているのでキャスト
  y.append(p_p)  # 正解ラベルを配列 y に追加 y = [ 0, 0, 0, 0, … ,8, 8, 8, 8 ] 

# 以下のfor 予測したいデータの名詞配列の数値化 or 辞書の生成
for word in p_N_dic:
    word = word.strip()  
    if word == "": continue
    if not word in dic: # 未登録の場合
      dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
      num = count
      count +=1
    # print(num,word)  # 数字と単語を表示
    else:
      num=dic[word] # 数字を辞書で調べる
    p_result.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している
    
#print(x)
#print(y)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
予測したい文章を入力してください。The number of undernourished people has dropped by almost half in the past two decades because of rapid economic growth and increased agricultural productivity. Many developing countries that used to suffer from famine and hunger can now meet their nutritional needs. Central and East Asia, Latin America and the Caribbean have all made huge progress in eradicating extreme hunger.Unfortunately, extreme hunger and malnutrition remain a huge barrier to development in many countries. There are 821 million people estimated to be chronically undernourished as of 2017, often as a direct consequence of environment

In [0]:
import numpy as np
import keras
import glob
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split ### 追加
from collections import Counter #カウントするメソッドを使うためのモジュール

max_words = len(dic)
batch_size = 150
epochs = 30

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=111)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
 
num_classes = np.max(y_train) +1
print(num_classes, 'classes')
 
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
 
print('Convert class vector to binary class matrix '
'(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

# modelのインスタンス生成
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))#２層ニューラルネットワーク

 
model.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']) 
 
history = model.fit(x_train, y_train,
batch_size=batch_size,
epochs=epochs,
verbose=1,
validation_split=0.1)
score = model.evaluate(x_test, y_test,
batch_size=batch_size, verbose=1)

pre=model.predict( x_test, batch_size=batch_size, verbose=1) #それぞれの正解ラベルに対する確率を返す

#予測データのバイナリー化
def binary(result):
  arr=np.zeros(len(dic)) #辞書数分のバイナリー配列を生成。すべて０で最初は生成
  for f in result: #予測したい文脈の名詞の番号のところに１を立てる。
    arr[f]=1

  arr=np.array(arr) #numpy配列に変換
  arr.reshape(1,len(dic))
  arrd=np.stack([arr, arr]) #無理やり配列を拡張。model.predictの引数に拡張しないと入れられないため。(2,len(dic))の配列にする
  return arrd
#print(arrd.shape)
bin_pre=binary(p_result)
pre1=model.predict( bin_pre, batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])
#print(pre1)

#出力関数
def result(pre):
  result= []
  for i,c in enumerate(pre[0]): #２次元配列を生成
    hum=[c,i]
    result.append(hum)
  result2=sorted(result,reverse=True) #大きい順にソート

  j=0
  for k,l in enumerate(result2):
    if l[1]==0:
      j=1
      continue
    if j==1:
      print(str(k)+"番目に関連が高いのは"+str(l[1])+"の目標で,関連度は"+str(l[0]*100)+"％です。")
    else:
      print(str(k+1)+"番目に関連が高いのは"+str(l[1])+"の目標で,関連度は"+str(l[0]*100)+"％です")
  return result2
current_result=result(pre1)

#推薦単語アルゴリズム
def renumbering(new_word,p_result): #推薦単語をすべて包括した場合の予測 任意の単語を追加して単語を数値化
  jaka=[]
  for word in new_word:
      word = word.strip()  
      if word == "": continue
      if not word in dic: # 未登録の場合
        dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
        num = count
        count +=1
      #print(num,word)  # 数字と単語を表示
      else:
        num=dic[word] # 数字を辞書で調べる
        jaka.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している
  return p_result+jaka

  
G=Counter(Cdic[current_result[0][1]]).most_common() #それぞれのゴール単語を包括した辞書を予測論文の１番関連度が高いゴールの単語を多い順に返してくれる。ここで型をリストに変えてくれる
print(G[0])

new_words=[] #表示させる推薦単語
for i,k in enumerate(G):
  if i==10: #上位の単語をいくつ調べるかの設定。
    break
  if k[0] in p_N_dic: #その論文に推薦単語がある場合処理をスキップ
    continue
  print("他に包括したほうが良い単語は"+k[0]+"です。\n") 
  new_words.append(k[0])
#G[0][0] [出現単語][出現回数]
new_predict=renumbering(new_words,p_result)
repredict=binary(new_predict) #2値化
pre2=model.predict( repredict, batch_size=batch_size, verbose=1) #予測データ取得
print("上記の単語を包括した場合の関連度は以下の様になります。")
result(pre2) #出力

print("その他のゴールも包括するには以下のような単語を推薦します")
for i in range(18):
  if i==0:
    continue
  if i==current_result[0][1]:
    continue
  G=Counter(Cdic[i]).most_common()
  print("Goal"+str(i)+"⇩")
  for i,k in enumerate(G):
    if i==10: #上位の単語をいくつ調べるかの設定。
      break
    if k[0] in p_N_dic: #その論文に推薦単語がある場合処理をスキップ
      continue
    print("他に包括したほうが良い単語は"+k[0]+"です。\n") 

### Plot accuracy
import matplotlib.pyplot as plt
acc = history.history["acc"]
val_acc = history.history["val_acc"]
epochs = range(1, len(acc) + 1)
 
plt.plot(epochs, acc, "bo", label = "Training acc" )
plt.plot(epochs, val_acc, "b", label = "Validation acc")
plt.title("Training and Validation accuracy")
plt.legend()
plt.savefig("acc.png")
plt.close()
 
### plot Confusion Matrix
import pandas as pd
import seaborn as sn
from sklearn.metrics import confusion_matrix
 
def print_cmx(y_true, y_pred):
  labels = sorted(list(set(y_true)))
  cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
 
  df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
  plt.figure(figsize = (10,7))
  sn.heatmap(df_cmx, annot=True, fmt="d") ### ヒートマップの表示仕様
  plt.title("Confusion Matrix")
  plt.xlabel("predict_classes")
  plt.ylabel("true_classes")
  plt.savefig("c_matrix.png")
  plt.close()
 
predict_classes = model.predict_classes(x_test[1:10000,], batch_size=32) ### 予測したラベルを取得
true_classes = np.argmax(y_test[1:10000],1) ### 実際のラベルを取得
print(confusion_matrix(true_classes, predict_classes))
print_cmx(true_classes, predict_classes)

268 train sequences
68 test sequences
18 classes
Vectorizing sequence data...
x_train shape: (268, 5071)
x_test shape: (68, 5071)
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (268, 18)
y_test shape: (68, 18)
Building model...
Train on 241 samples, validate on 27 samples
Epoch 1/30
241/241 [==============================] - 1s 4ms/step - loss: 2.9171 - acc: 0.0996 - val_loss: 2.7058 - val_acc: 0.2963
Epoch 2/30
241/241 [==============================] - 0s 582us/step - loss: 2.6515 - acc: 0.2614 - val_loss: 2.5311 - val_acc: 0.2963
Epoch 3/30
241/241 [==============================] - 0s 588us/step - loss: 2.3674 - acc: 0.4149 - val_loss: 2.3506 - val_acc: 0.2593
Epoch 4/30
241/241 [==============================] - 0s 577us/step - loss: 2.0592 - acc: 0.4813 - val_loss: 2.2196 - val_acc: 0.2222
Epoch 5/30
241/241 [==============================] - 0s 583us/step - loss: 1.8301 - acc: 0.4523 - val_loss: 2.1768 - val_acc: 0.2222
Epoch 6

In [0]:
result2[1][1]

NameError: ignored

In [0]:
"""
入力用データ成型機
"""
#https://qiita.com/shimayu22/items/4fc6d01a4bf0eef909a0 参考記事
#https://qiita.com/shimayu22/items/4fc6d01a4bf0eef909a0
#課題～ハイパーパラメータをいじる、あるいはデータの抜き出し、整形の制度を上げる。
#数値のベクトル化のところのアルゴリズムが少し不自然な気がするからそこの修正
#ネット回線ないとコード動きません。
#ネットのSDGs用語も辞書に足し合わせるべし←ネットに乗っている用語は重要度を上げるなどの改良の余地あり。
#大文字を小文字にする処理や記号を抜き取る処理を追加する。
import os
import csv
import nltk
from sklearn.model_selection import train_test_split

nltk.download('punkt') #nltkを使うためのソフト群
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

csv_file = open(data_dir,'r') #学習データ
p_n_list = input("予測したい文章を入力してください。")
#p_csv_file= open(predict_dir,"r") #予測したいデータ
"""
csvファイルは左から目標番号、タイトル、アブスト、結論の順で入れること。
"""
p_list,t_list,a_list = [],[],[]
#c_list= []

delete= ["a","%","of","in","on","to","for","are","at","[","]"] #関連性のない語を集めた配列

#p_n_list=[]#予測したいデータのテキストバッグ

#for row in csv.reader(p_csv_file):
#  p_n_list.append(row[1])
#  p_n_list.append(row[2])

x, y = [], []
count = 0

for row in csv.reader(csv_file):
  #以下リストを作成
    p_list.append(row[0]) #取得したい列番号を指定（0始まり）#目標番号をすべて取得
    t_list.append(row[1]) #タイトルをすべて取得
    a_list.append(row[2]) #アブストをすべて取得
    #c_list.append(row[3]) #結論も入れる

del p_list[0] #0番目を削除
del t_list[0] #0番目は削除
del a_list[0] #0番目は削除。これで_list[i]でそれぞれの列を抜き出すことができる。
#del c_list[0]

#以下３行　予測したいデータの形態素解析
#p_n_list=p_n_list[0]+p_n_list[1] 
p_n_words = nltk.word_tokenize(p_n_list)
p_n_wards=nltk.pos_tag(p_n_words)#[("単語","品詞")....]の２次元リストをここで生成


p_result = [] #名詞の数字配列を入れるp_resultを定義
p_N_dic = [] #名詞のstring型配列を定義
#以下名詞を抜き出しN_dicに入れ込み
for word,part in p_n_wards: #wordに単語、partに品詞が入っている→このforはタイトル
    if part=="NN" or part=="NNP" or part=="VBP": #以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete: #さらにdeleteリストに入っている意味のない名詞を削除
        sword=word.lower()
        p_N_dic.append(sword)

Cdic=[] #各ゴールの単語をすべて入れる。推薦単語用.Cdic[1]→1番目の単語すべてを返す

for i in range(18): #それぞれのゴールの中身を内包した収納配列を生成
  a=[]
  Cdic.append(a) #[[],[],[]......18個]この中に単語をすべて入れていく

dic={} #辞書で定義
for i,p in enumerate(p_list): #p_listのiをカウンタ変数,ｐを正解ラベル
  j=i+1 #p_list,a_listなどは0番目が存在しないため無理やりjをカウンター変数に置き換え
  if j>=len(p_list): #+1してるから最後の一つだけ存在しないjが入るためここで分岐処理　t_list[j_max]は存在しないためエラー
    break
  N_dic=[] #ここで名詞リストは初期化
  #print(t_list[i])
  t_string = t_list[j]
  t_words = nltk.word_tokenize(t_string)
  t_wards=nltk.pos_tag(t_words)#[("単語","品詞")....]の２次元リストをここで生成
  #print(t_wards)
  a_string = a_list[j]
  a_words = nltk.word_tokenize(a_string)
  a_wards=nltk.pos_tag(a_words)
  """
  c_string = c_list[j]
  c_words = nltk.word_tokenize(c_string)
  c_wards=nltk.pos_tag(c_words)
  """
  for word,part in t_wards: #wordに単語、partに品詞が入っている→このforはタイトル
    if part=="NN" or part=="NNP" or part=="VBP" or part=="NNS" or part=="NNPS": #以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete: #さらにdeleteリストに入っている意味のない名詞を削除
        sword=word.lower() #小文字に変換
        N_dic.append(sword)
        Cdic[int(p)].extend([sword]) #swordはstr型なので無理やりリストに変換 extendはlist型しか入らない
  for word,part in a_wards: #wordに単語、partに品詞が入っている→このforはアブスト
    if part=="NN" or part=="NNP" or part=="VBP" or part=="NNS" or part=="NNPS": #以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete:
        sword=word.lower()
        N_dic.append(sword)
        Cdic[int(p)].extend([sword]) #pはstrなのでキャスト
  """
  for word,part in c_wards: #wordに単語、partに品詞が入っている→このforはアブスト
    if part=="NN": #以下if文は名詞のみをN_dicというリストに格納
      N_dic.append(word)
    if part=="NNP":
      N_dic.append(word)
  """
  print(N_dic)#すべてN_dicにまとめてしまう。

  result=[]
  for word in N_dic:
    word = word.strip()  
    if word == "": continue
    if not word in dic: # 未登録の場合
      dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
      num = count
      count +=1
    #print(num,word)  # 数字と単語を表示
    else:
      num=dic[word] # 数字を辞書で調べる
    result.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している
  x.append(result)  # リストを配列 ｘ に追加 x = [ [ 10, 11, 12, 3, 8, 13,  … 38 ] , [ 39, 40, 12, 16, 19, … ,56 ] , ….  ,  [ 1200, 1504, 3, 15, … 3300 ] ] 
  p_p=int(p)  #pは文字型で入っているのでキャスト
  y.append(p_p)  # 正解ラベルを配列 y に追加 y = [ 0, 0, 0, 0, … ,8, 8, 8, 8 ] 

#以下のfor 予測したいデータの名詞配列の数値化 or 辞書の生成
for word in p_N_dic:
    word = word.strip()  
    if word == "": continue
    if not word in dic: # 未登録の場合
      dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
      num = count
      count +=1
    #print(num,word)  # 数字と単語を表示
    else:
      num=dic[word] # 数字を辞書で調べる
    p_result.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している
    
print(x)
print(y)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
予測したい文章を入力してください。By 2030, ensure that all young people and the majority (both men and women) adults have literacy and basic computing skills. By 2030, education for sustainable development and sustainable lifestyles, human rights, gender equality, promotion of peace and non-violent culture, global citizenship, sustainable development of cultural diversity and culture Ensure that all learners have the knowledge and skills necessary to promote sustainable development through education in understanding their contributions. Establish and improve educational facilities that are child,

In [0]:
#バイナリー化の証明
#x2a=x_test[2]
print(x2a)
print(x_test[2])

arr2=np.zeros(5242)

for f in x2a:
  arr2[f]=1

print(arr2)
#arr=p_resut

[1055, 2413, 209, 2414, 1622, 2415, 2416, 2417, 2418, 2419, 0, 2420, 2421, 2422, 2423, 2404, 2424, 2425, 97, 2426, 257, 2427, 2428, 2426, 2429, 2430, 97, 2431, 281, 2432, 169, 2426, 2432, 2433, 2434, 2435, 794, 117, 505, 239, 2436, 2437, 2436, 2438, 2436, 2432, 30, 2436, 239, 28, 2432, 2432]
[1. 0. 0. ... 0. 0. 0.]


In [0]:

print(dic)
print(Cdic)

{'development': 0, 'cost': 1, 'water-level': 2, 'sensor': 3, 'paddy': 4, 'field': 5, 'embedded': 6, 'system': 7, 'iot': 8, 'water': 9, 'level': 10, 'control': 11, 'rice': 12, 'farming': 13, 'lot': 14, 'time': 15, 'efforts': 16, 'number': 17, 'fields': 18, 'farmers': 19, 'have': 20, 'futhermore': 21, 'techniques': 22, 'trol': 23, 'exprience': 24, 'situations': 25, 'people': 26, 'internet': 27, 'need': 28, 'research': 29, 'waterlevel': 30, 'propose': 31, 'attempt': 32, 'optimized': 33, 'sewage': 34, 'sludge': 35, 'compost': 36, 'button': 37, 'mushroom': 38, 'cultivation': 39, 'japan': 40, 'tons': 41, 'treatment': 42, 'plants': 43, 'construction': 44, 'materials': 45, 'content': 46, 'accounts': 47, 'total': 48, 'utilization': 49, 'ratio': 50, 'characteristics': 51, 'contents': 52, 'group': 53, 'biomass': 54, 'oyster': 55, 'mushrooms': 56, 'amino': 57, 'acids': 58, 'productivity': 59, 'methods': 60, 'study': 61, 'difference': 62, 'community': 63, 'compositions': 64, 'order': 65, 'bacillale

In [0]:
#高次元化
a=["aaa"]+["ddsa","dsa"]
a

['aaa', 'ddsa', 'dsa']

In [0]:
'''
for j,word in enumerate(t_words)
  #string = "I AM A CAT. As yet I have no name."
  words = nltk.word_tokenize(t_string) #分かち書きをここで行っている→それぞれの単語をリスト化
  wards=nltk.pos_tag(words) #品詞と単語の２次元リストで返す関数 ex)wards[x][y]→NNP
  wards[1]
  string = a_list[0]
  #string = "I AM A CAT. As yet I have no name."
  words = nltk.word_tokenize(string) #分かち書きをここで行っている→それぞれの単語をリスト化
  wards=nltk.pos_tag(words) #品詞と単語の２次元リストで返す関数 ex)wards[x][y]→NNP
  wards[1]
'''
#https://www.haya-programming.com/entry/2018/03/21/234126
#英語版の形態素解析
import nltk
nltk.download('punkt')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
string = "I AM A CAT. As yet I have no name."
words = nltk.word_tokenize(string)
words
wards=nltk.pos_tag(words)
wards[1]
例題
4
By 2030, ensure that all young people and the majority (both men and women) adults have literacy and basic computing skills.
By 2030, education for sustainable development and sustainable lifestyles, human rights, gender equality, promotion of peace and non-violent culture, global citizenship, sustainable development of cultural diversity and culture Ensure that all learners have the knowledge and skills necessary to promote sustainable development through education in understanding their contributions.
Establish and improve educational facilities that are child, disability and gender sensitive and provide a safe, non-violent, inclusive and effective learning environment for all.
By 2020, developed countries such as vocational training, information and communication technology (ICT), technology, engineering and science programs for developing countries, especially least developed and small island developing countries, and African countries Significantly increase the number of higher education scholarships worldwide in other developing countries.

5
Eliminate all forms of discrimination against all women and girls everywhere.
5.2
Eliminate all forms of violence in public and private spaces against all women and girls, including human trafficking, sexual and other types of exploitation.
5.3
Eliminate all harmful practices such as minor marriage, early marriage, forced marriage and genital mutilation.
5.4
Recognize and evaluate unpaid childcare / nursing care and domestic labor through provision of public services, infrastructure and social security policies, and sharing of responsibilities among households and families according to the situation in each country.
5.5
Ensure full and effective women's participation and equal leadership opportunities at all levels of decision-making in politics, economy and public sector.
5.6
Ensure universal access to sexual and reproductive health and rights in accordance with the International Population and Development Council (ICPD) Action Plan and Beijing Code of Conduct, and the outcome documents of these verification meetings.

6
By 2030, achieve universal and equal access to safe and affordable drinking water for all
6.2
By 2030, ensure access to adequate and equitable sewage and sanitation for all people and eliminate open defecation Pay particular attention to the needs of women and girls and vulnerable people.
6.3
By 2030, reducing pollution, eliminating dumping and minimizing the release of hazardous chemicals and substances, halving the proportion of untreated wastewater, and significantly increasing recycling and safe reuse on a global scale To improve water quality.
6.4
By 2030, drastically improve water use efficiency in all sectors, ensure sustainable collection and supply of fresh water, address water shortages, and significantly reduce the number of people suffering from water shortages.
6.5
By 2030, implement integrated water resources management at all levels, including through transboundary cooperation as appropriate.
6.6
By 2020, protect and restore water-related ecosystems, including mountains, forests, wetlands, rivers, aquifers, and lakes.

#https://qiita.com/shimayu22/items/4fc6d01a4bf0eef909a0 参考記事
#https://qiita.com/shimayu22/items/4fc6d01a4bf0eef909a0
#課題～ハイパーパラメータをいじる、あるいはデータの抜き出し、整形の制度を上げる。
#数値のベクトル化のところのアルゴリズムが少し不自然な気がするからそこの修正
#ネット回線ないとコード動きません。
#ネットのSDGs用語も辞書に足し合わせるべし←ネットに乗っている用語は重要度を上げるなどの改良の余地あり。
#大文字を小文字にする処理や記号を抜き取る処理を追加する。
import os
import csv
import nltk
from sklearn.model_selection import train_test_split

nltk.download('punkt') #nltkを使うためのソフト群
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')

csv_file = open(data_dir,'r') #学習データ
p_csv_file= open(predict_dir,"r") #予測したいデータ
"""
csvファイルは左から目標番号、タイトル、アブスト、結論の順で入れること。
"""
p_list,t_list,a_list=[],[],[]
#c_list= []

delete= ["a","%","of","in","on","to","for","are","at"] #関連性のない語を集めた配列

p_n_list=[]#予測したいデータのテキストバッグ

for row in csv.reader(p_csv_file):
  p_n_list.append(row[1])
  p_n_list.append(row[2])

x, y = [], []
count = 0

for row in csv.reader(csv_file):
  #以下リストを作成
    p_list.append(row[0]) #取得したい列番号を指定（0始まり）#目標番号をすべて取得
    t_list.append(row[1]) #タイトルをすべて取得
    a_list.append(row[2]) #アブストをすべて取得
    #c_list.append(row[3]) #結論も入れる

del p_list[0] #0番目を削除
del t_list[0] #0番目は削除
del a_list[0] #0番目は削除。これで_list[i]でそれぞれの列を抜き出すことができる。
#del c_list[0]

#以下３行　予測したいデータの形態素解析
p_n_list=p_n_list[0]+p_n_list[1] 
p_n_words = nltk.word_tokenize(p_n_list)
p_n_wards=nltk.pos_tag(p_n_words)#[("単語","品詞")....]の２次元リストをここで生成


p_result = [] #名詞の数字配列を入れるp_resultを定義
p_N_dic = [] #名詞のstring型配列を定義
#以下名詞を抜き出しN_dicに入れ込み
for word,part in p_n_wards: #wordに単語、partに品詞が入っている→このforはタイトル
    if part=="NN" or part=="NNP" or part=="VBP": #以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete: #さらにdeleteリストに入っている意味のない名詞を削除
        sword=word.lower()
        p_N_dic.append(sword)

dic={} #辞書で定義
for i,p in enumerate(p_list): #p_listのiをカウンタ変数,ｐを正解ラベル
  j=i+1 #p_list,a_listなどは0番目が存在しないため無理やりjをカウンター変数に置き換え
  if j>=len(p_list): #+1してるから最後の一つだけ存在しないjが入るためここで分岐処理　t_list[j_max]は存在しないためエラー
    break
  N_dic=[] #ここで名詞リストは初期化
  #print(t_list[i])
  t_string = t_list[j]
  t_words = nltk.word_tokenize(t_string)
  t_wards=nltk.pos_tag(t_words)#[("単語","品詞")....]の２次元リストをここで生成
  #print(t_wards)
  a_string = a_list[j]
  a_words = nltk.word_tokenize(a_string)
  a_wards=nltk.pos_tag(a_words)
  """
  c_string = c_list[j]
  c_words = nltk.word_tokenize(c_string)
  c_wards=nltk.pos_tag(c_words)
  """
  for word,part in t_wards: #wordに単語、partに品詞が入っている→このforはタイトル
    if part=="NN" or part=="NNP" or part=="VBP": #以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete: #さらにdeleteリストに入っている意味のない名詞を削除
        sword=word.lower() #小文字に変換
        N_dic.append(sword)
  for word,part in a_wards: #wordに単語、partに品詞が入っている→このforはアブスト
    if part=="NN" or part=="NNP" or part=="VBP": #以下if文は名詞のみをN_dicというリストに格納
      if not word.lower() in delete:
        sword=word.lower()
        N_dic.append(sword)  
  """
  for word,part in c_wards: #wordに単語、partに品詞が入っている→このforはアブスト
    if part=="NN": #以下if文は名詞のみをN_dicというリストに格納
      N_dic.append(word)
    if part=="NNP":
      N_dic.append(word)
  """
  print(N_dic)#すべてN_dicにまとめてしまう。

  result=[]
  for word in N_dic:
    word = word.strip()  
    if word == "": continue
    if not word in dic: # 未登録の場合
      dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
      num = count
      count +=1
    #print(num,word)  # 数字と単語を表示
    else:
      num=dic[word] # 数字を辞書で調べる
    result.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している
  x.append(result)  # リストを配列 ｘ に追加 x = [ [ 10, 11, 12, 3, 8, 13,  … 38 ] , [ 39, 40, 12, 16, 19, … ,56 ] , ….  ,  [ 1200, 1504, 3, 15, … 3300 ] ] 
  p_p=int(p)  #pは文字型で入っているのでキャスト
  y.append(p_p)  # 正解ラベルを配列 y に追加 y = [ 0, 0, 0, 0, … ,8, 8, 8, 8 ] 

#以下のfor 予測したいデータの名詞配列の数値化 or 辞書の生成
for word in p_N_dic:
    word = word.strip()  
    if word == "": continue
    if not word in dic: # 未登録の場合
      dic[word] = count  # count の数字で辞書に登録　補足：辞書型はwordとkeyの二つの情報を入れ込むことができる.これでNNに入れれるようにしていく
      num = count
      count +=1
    #print(num,word)  # 数字と単語を表示
    else:
      num=dic[word] # 数字を辞書で調べる
    p_result.append(num)  # リストに数字を追加 ここで名詞の塊を数値化している

print(x)
print(y)

In [0]:
import nltk
nltk.download('punkt')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
string = "By 2030, ensure that all young people and the majority (both men and women) adults have literacy and basic computing skills."
words = nltk.word_tokenize(string)
words
wards=nltk.pos_tag(words)
for i in range(len(wards)):
  print(wards[i])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
('By', 'IN')
('2030', 'CD')
(',', ',')
('ensure', 'VB')
('that', 'IN')
('all', 'DT')
('young', 'JJ')
('people', 'NNS')
('and', 'CC')
('the', 'DT')
('majority', 'NN')
('(', '(')
('both', 'DT')
('men', 'NNS')
('and', 'CC')
('women', 'NNS')
(')', ')')
('adults', 'NNS')
('have', 'VBP')
('literacy', 'NN')
('and', 'CC')
('basic', 'JJ')
('computing', 'VBG')
('skills', 'NNS')
('.', '.')


In [0]:
#推薦単語アルゴリズム検証
from collections import Counter #カウントするメソッドを使うためのモジュール
l=["apple","orange","orange","banana","apple","lemon","apple","frape"]
G=Counter(Cdic[2]).most_common() #多い順に返してくれる。ここで型をリストに変えてくれる
print(G)
L=G[0]
print(L)


[('system', 26), ('nematodes', 26), ('compost', 24), ('sludge', 22), ('reactor', 18), ('water', 17), ('sewage', 17), ('study', 16), ('bacillus', 13), ('cultivation', 11), ('l', 11), ('field', 10), ('mushroom', 9), ('soil', 9), ('root', 9), ('disease', 9), ('flow', 9), ('exchange', 9), ('development', 8), ('button', 8), ('lotus', 8), ('dbd', 7), ('plasma', 7), ('color', 7), ('characteristics', 6), ('community', 6), ('order', 6), ('types', 6), ('gene', 6), ('production', 6), ('plant', 6), ('blood', 6), ('ldv', 6), ('wastewater', 6), ('abr', 6), ('o', 6), ('aquaria', 6), ('cost', 5), ('sensor', 5), ('control', 5), ('time', 5), ('have', 5), ('plants', 5), ('bacteria', 5), ('diseases', 5), ('day', 5), ('technology', 5), ('application', 5), ('groups', 5), ('days', 5), ('performance', 5), ('operation', 5), ('information', 5), ('level', 4), ('rice', 4), ('attempt', 4), ('japan', 4), ('treatment', 4), ('ratio', 4), ('mushrooms', 4), ('productivity', 4), ('composts', 4), ('condition', 4), ('anal

In [0]:
a=[]
d=["dsa","ads","ww"]
b=[]
c=[]
c.append(a)
c.append(b)
c[1].extend(d)
c
for i range(16)
  a=[]
  

[[], ['dsa', 'ads', 'ww']]

In [0]:
c=[]
for i in range(16):
  a=[]
  c.append(a)
d="wdsw"
ss=["ds","dsss","daas"]
c[4].extend([d])
c[4].extend(ss)
print(c)

[[], [], [], [], ['wdsw', 'ds', 'dsss', 'daas'], [], [], [], [], [], [], [], [], [], [], []]


In [0]:
def counts(a,b):
  print("aaa"+str(a))



In [0]:
counts(1,1)

aaa1
